# **Pseudopotential**

<i class="fa fa-home fa-2x"></i><a href="./index.ipynb" style="font-size: 20px"> Go back to index</a>

**Source code:** https://github.com/osscar-org/quantum-mechanics/blob/develop/notebook/pseudopotential.ipynb

<hr style="height:1px;border:none;color:#cccccc;background-color:#cccccc;" />

## **Goals**

<p style="text-align: justify;font-size:15px">
    This notebook presents how pseudopotentials are constructed. The pseudopotential 
    is an effective potential constructed to replace the complicated effects of the 
    core electrons. We used the theory and method from the book 
    <a href="https://www.sciencedirect.com/book/9780123044600/solid-state-physics">
    "Soild State Physics (Second Edition)"</a>
    by Giuseppe Grosso and Giuseppe Pastori Parravicini.
</p>

<details close>
    <summary style="font-size: 20px"><b>Sub-goals</b></summary>
    <ol style="text-align: justify;font-size:15px">
        <li> Understand the free-electron model. </li>
        <li> Examine the electronic band structure for different cells. </li>
        <li> Know how to construct the Brillouin zone. </li>
        <li> Understand Monkhorst-Pack k-points. </li>
        <li> Examine the density of states. </li>
        <li> Describe how to calculate the density of states. </li>
    </ol>

</details>

## **Theory: construct pseudopotential**

<p style="text-align: justify;font-size:15px">
    The raidal Schrödinger equation is given as:
</p>

$\large \left[
  -\dfrac{1}{2m} \, \dfrac{\partial^2}{\partial r^{2}} + 
  \dfrac{l(l+1)}{2r^2} + V(r)\right] r\psi(r) = Er\psi(r)$ (1)
  
<p style="text-align: justify;font-size:15px">
    The top figure in the below shows the $\Psi(r)r$ and pseudowavefunction $\Psi^{PS}(r)r$.
    We need choose a resonable cutoff distance to sperate the core region and the rest.
    You can use the slider to change the cutoff distance $R_c$. We need to replace the 
    core region wavefunction with a smoonth and nodeless function. Here, we construct the
    pseudo wavefunction as:
</p>


$ \large  \Psi^{pseudo}(r) =
    \begin{cases}
      r^le^{p(r)}  & \text{for r < r$_c$} \\
      \Psi(r) & \text{for r > r$_c$}
    \end{cases}       
$

<p style="text-align: justify;font-size:15px">
    where $p(r)$ is a polynomial defined as:
</p>

$ \large p(r) = \lambda_0 + \sum_{2}^{n} \lambda_n r
$

<p style="text-align: justify;font-size:15px">
    In this polynomial, the 1st order coefficient $\lambda_1$ is set to be zero, 
    which presents the pseudopotential being singular at $r=0$. In this notebook,
    we choice to use the 4th order polynomial.
</p>

$ \large p(r) = \lambda_0 + \lambda_2 r^2 + \lambda_3 r^3 + \lambda_4 r^4 $

## **Conditions for the pseudo wavefunction**

<p style="text-align: justify;font-size:15px">
    We need four equations to obtain the coefficients ($\lambda$). The equations
    are formulated with two conditions.
</p>

### **1. continuity of function at r$_c$**

<p style="text-align: justify;font-size:15px">
    The pseudo wavefunction needs keep continuity at r$_c$. The zero, 1st and
    2nd left derivatives at r$_c$ should equal the respective derivatives at
    right. Here, we can formulate three equations.
</p>

### **2. Norm-Conserving**

<p style="text-align: justify;font-size:15px">
    The norms of the wavefunction and pseudo wavefunction should be the same.
    Since when $r > r_c$ the functions are the same, we only need to integrate
    from zero to r$_c$.
</p>

$\Large \int_0^{r_c} \Psi(r)^2r^2 dr = \int_0^{r_c} \Psi^{PS}(r)^2r^2 dr
$

<p style="text-align: justify;font-size:15px">
    which also means the yellow area as shown in the figure below should be
    equal to the green area.
</p>

## **Obtain the pseudopotential**

<p style="text-align: justify;font-size:15px">
    In the final step, the pseudopotential is computed from the inversed Shrödinger equation.
</p>

$\large V^{PS}(r) = E - \dfrac{l(l+1)}{2r^2} + \dfrac{1}{2\Psi^{PS}(r)}
\dfrac{\partial^2 \Psi^{PS}(r)}{\partial r^{2}}$

In [ ]:
from sympy.physics.hydrogen import R_nl, E_nl
from sympy.abc import r
from sympy.functions import exp
import numpy as np
from sympy import *
import matplotlib.pyplot as plt
from sympy.solvers.solveset import linsolve
from ipywidgets import FloatSlider, Button, IntSlider
from scipy.optimize import newton
import matplotlib.gridspec as gridspec


%matplotlib widget

In [ ]:
n = 3
l = 0

ho = R_nl(n, l, r, Z=1)
Ea = E_nl(n, Z=1)
rf = lambdify(r, ho, "numpy")

In [ ]:
s_rc = FloatSlider(value = 30.0, min = 0.0, max = 70, description = "Cutoff $R_c$ : ", 
                   layout={'width':'600px'});

compute = Button(description="Compute pseudopotential");


img = plt.figure(tight_layout=True, figsize=(8,7))
img.canvas.header_visible = False

gs = gridspec.GridSpec(2, 1)

ax1 = img.add_subplot(gs[0, 0])
ax2 = img.add_subplot(gs[1, 0])


x1 = np.arange(0, 70.0, 0.01)
y1 = rf(x1)*x1

ax1.plot(x1, y1, 'r-', label="$\Psi(r)r$")
ax1.fill_between(x1, y1, 0, where=x1<s_rc.value, facecolor='yellow', alpha=0.5)
ax1.set_xlim([0, 70.0])

ax1.hlines(0, 0, 70, 'k','--')

line_rc1 = ax1.axvline(s_rc.value)
line_pswf, = ax1.plot([],[],'b-', linewidth=1.5, label="$\Psi^{PS}(r)r$")
ann_rc = ax1.annotate("$R_c$", xy=(s_rc.value + 1.0, rf(s_rc.value)*s_rc.value), fontsize=20)
point, = ax1.plot(s_rc.value, rf(s_rc.value)*s_rc.value, 'ko')

ann_norm1 = ax1.annotate("Yellow area:", xy=(40, -0.1), fontsize=12, 
                         bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5))
ann_norm2 = ax1.annotate("Green area:", xy=(40, -0.15), fontsize=12,
                         bbox=dict(boxstyle='round', facecolor='green', alpha=0.5))

ax1.set_xlabel("r", fontsize = 15)
ax1.set_ylabel("$\Psi(r)r$", fontsize = 15)

ax1.legend(fontsize=15)


x2 = np.linspace(5.0, 70, 600);
y2 = -1.0/x2;

ax2.plot(x2, y2, 'r--', linewidth=1.0, label="$-Z/r$")

ax2.set_xlim([0, 70.0])
ax2.set_ylim([-0.10, 0.10])
line_rc2 = ax2.axvline(s_rc.value)
line_psv, = ax2.plot([],[], 'b-', linewidth=1.5, label="$V^{SP}(r)$")
ax2.hlines(0, 0, 70, 'k','--')

ax2.set_xlabel("r", fontsize = 15)
ax2.set_ylabel("$V$", fontsize = 15)

ax2.legend(fontsize=15)

plt.show()

def on_rc_change(b):
    line_rc1.set_data(s_rc.value, [-1, 1])
    line_rc2.set_data(s_rc.value, [-1, 1])
    ann_rc.set_position((s_rc.value + 1.0, rf(s_rc.value)*s_rc.value))
    ax1.collections.clear()
    ax1.fill_between(x1, y1, 0, where=x1<s_rc.value, facecolor='yellow', alpha=0.5)
    ax1.hlines(0, 0, 70, 'k','--')
    line_pswf.set_data([],[])
    line_psv.set_data([],[])
    point.set_data(s_rc.value, rf(s_rc.value)*s_rc.value)
    
    
s_rc.observe(on_rc_change, names='value')

display(s_rc)

In [ ]:
def compute_right_derivative(rc):
    k0 = rf(rc)
    k1 = diff(ho, r).subs(r, rc).evalf()
    k2 = diff(ho, r, 2).subs(r, rc).evalf()
    return np.array([float(k0), float(k1), float(k2)])

def compute_left_derivative(ps, rc):
    psf = lambdify(r, ps, "numpy")
    k0 = psf(rc)
    k1 = diff(ps, r).subs(r, rc).evalf()
    k2 = diff(ps, r, 2).subs(r, rc).evalf()
    return np.array([float(k0), float(k1), float(k2)])
    
def solver_kernel(devs, rc, l, b):
    A = np.zeros([3,4])
    A[0, :] = np.array([1, rc**2, rc**3, rc**4]);
    A[1, :] = np.array([0, 2*rc, 3*rc**2, 4*rc**3]);
    A[2, :] = np.array([0, 2, 6*rc, 12*rc**2])
    
    B = np.zeros(3)
    B[0] = log(devs[0]/rc**l);
    B[1] = devs[1]/devs[0] - l/rc;
    B[2] = devs[2]/devs[0] - devs[1]**2/devs[0]**2 + l/rc**2;
    
    B-=b*A[:, 1];
    A = np.delete(A, (1), axis=1);
    
    coff = np.linalg.solve(A, B);
    coff = np.insert(coff, 1, b)
    
    return coff

def diff_norms(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    ps  = r**l*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = lambdify(r, ps, "numpy")
    psr = lambdify(r, ps*ps*r*r, "numpy")
    hor = lambdify(r, ho*ho*r*r, "numpy")
        
    x1 = np.linspace(0, rc, 800);
        
    norm1 = np.sum(x1*hor(x1))*(x1[1]-x1[0])
    norm2 = np.sum(x1*psr(x1))*(x1[1]-x1[0])
    
    return float(norm1 - norm2)

def compute_norms(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    ps  = r**l*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = lambdify(r, ps, "numpy")
    psr = lambdify(r, ps*ps*r*r, "numpy")
    hor = lambdify(r, ho*ho*r*r, "numpy")
        
    x1 = np.linspace(0, rc, 800);
        
    norm1 = np.sum(x1*hor(x1))*(x1[1]-x1[0])
    norm2 = np.sum(x1*psr(x1))*(x1[1]-x1[0])
    
    return norm1, norm2

def compute_potential(l):
    psf = Ea - l*(l+1)/(2*r*r) + 1/(2*ho*r)*diff(ho*r, r, r)
    return lambdify(r, psf, "numpy")


def plot_ps_wavefunction(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    ps  = r**l*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = lambdify(r, ps*r, "numpy")
    
    devl = compute_left_derivative(ps, rc)
    
    x1 = np.linspace(0, rc, 800);
    line_pswf.set_data(x1, psf(x1));
    ax1.fill_between(x1, psf(x1), 0, where=x1<s_rc.value, facecolor='green', alpha=0.5)

def plot_ps_potential(b, rc, l):
    devs = compute_right_derivative(rc)
    coff = solver_kernel(devs, rc, l, b)
    
    pf  = r**(l+1)*exp(coff[0] + coff[1]*r**2 + coff[2]*r**3 + coff[3]*r**4)
    psf = Ea - l*(l+1)/(2*r*r) + 1/(2*pf)*diff(pf, r, r)
    psfnl = lambdify(r, psf, "numpy")
    psfnr = compute_potential(l)
        
    x1 = np.linspace(0.1, rc, 800);
    x2 = np.linspace(rc, 70, 800);
    line_psv.set_data(np.concatenate((x1,x2)), np.concatenate((psfnl(x1),psfnr(x2))));

def compute_pseudopotential(c):
    b = newton(lambda x: diff_norms(x, s_rc.value, l), x0 = 0.0)
    plot_ps_wavefunction(b, s_rc.value, l)
    plot_ps_potential(b, s_rc.value, l)
    norm1, norm2 = compute_norms(b, s_rc.value, l)
    ann_norm1.set_text("Yellow area:" + str("{:.10f}".format(norm1)))
    ann_norm2.set_text("Green area:" + str("{:.10f}".format(norm2)))
    
compute.on_click(compute_pseudopotential)
compute_pseudopotential("init");

display(compute)